# 1.2-agifford-AnalyzeSingleDataFileAndSplitFiles
This notebook performs exploratory data analysis on an example datafile. Specifically, we identify peak frequencies as a function of activity label and attempt to determine an appropriate threshold level on the magnitude of the peaks to identify those frequencies to use as features in a model for predicting activity. We want a threshold that yields only ~1-2 peak frequencies per activity (not including the 0-frequency average).

This notebook also templates out code to sort raw parquet files into train/val vs. test data sets so that, moving forward, we will only explore files within the training set and use the validation set strictly for tweaking the model and the test set strictly for evaluating the model.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.fftpack import fft, fftshift
from pathlib import Path
from collections import defaultdict
import re

parq_file = "../../data/interim/raw/fileID1_subjID3_dataID0.parquet"
df = pd.read_parquet(parq_file, engine="fastparquet")

In [ ]:
def _make_single_annot_frame(df, shift):
    annot_df = df[df.label != df.label.shift(shift)]
    annot_df = annot_df.dropna(subset="label").reset_index()
    return annot_df

def make_annot_dataframe(df, t_start=None, t_end=None):
    t_start = t_start or df.time.min()
    t_end = t_end or df.time.max()
    
    df = df[(df.time >= t_start) & (df.time <= t_end)].copy()
    
    (act_starts_df, act_ends_df) = (
        _make_single_annot_frame(df, shift) for shift in [1, -1]
    )
    return act_starts_df, act_ends_df

In [ ]:
# not sure I'll need the starts & ends df, but probably should remove the rows with no 
# activity labels
activity_starts_df, activity_ends_df = make_annot_dataframe(df)
df_dropna =  df.dropna(subset="label")

In [ ]:
df.shape, df_dropna.shape

For the sake of getting through this project end-to-end, I will not spend too much time on building a very sophisticated model. As such, I will use the column `label_group` as my desired prediction column.

Process for each `label_group`:
1. Compute FFT with a Hanning window for each instance of the `label_group`
2. Average the FFTs across instances
3. Identify the major frequencies of the group by setting some arbitrary threshold to identify peaks.
4. I will use those frequencies to generate `sin` and `cos` features as inputs to a basic model to predict `label_group`.
5. Repeat steps 1-4 for each variable x direction combination (e.g., "accel_x", "accel_y", etc.)

First, let's template out the process of analyzing a single instance of a single `label_group`.

In [ ]:
act_tp_df = pd.concat([activity_starts_df.head(1), activity_ends_df.head(1)], ignore_index=True)
df_snip = df_dropna[(df_dropna.time >= act_tp_df.loc[0, "time"]) & (df_dropna.time <= act_tp_df.loc[1, "time"])]

In [ ]:
fs = 50
n_fft = df_snip.shape[0]
window = signal.hann(n_fft)
X_w = fft(window * df_snip.accel_x.values)
n_points = 2 * int(np.floor(n_fft / 2))
if n_fft % 2:
    n_points += 1
freq = fs/2 * np.linspace(-1, 1, n_points)


Nothing strong in "\<Initial Activity\>" except for 0 Hz...

In [ ]:
# X_w_norm = np.abs(fftshift(X_w))
X_w_norm = 20 * np.log10(np.abs(fftshift(X_w / abs(X_w).max())))
plt.plot(freq, X_w_norm)
plt.title("Frequency response first activity")
plt.ylabel("Normalized magnitude [dB]")
plt.xlabel("F [Hz]")
print(act_tp_df.loc[0, "label"])
plt.show()

In [ ]:
act_tp_df = pd.concat([activity_starts_df.loc[[4], :], activity_ends_df.loc[[4], :]], ignore_index=True)
df_snip = df_dropna[(df_dropna.time >= act_tp_df.loc[0, "time"]) & (df_dropna.time <= act_tp_df.loc[1, "time"])]

In contrast, there seem to be many prevalent peaks in "Jumping Jacks" at ~1 Hz and 2.75Hz.

In [ ]:
fs = 50
n_fft = df_snip.shape[0]
window = signal.hann(n_fft)
X_w = fft(window * df_snip.accel_x.values)
n_points = 2 * int(np.floor(n_fft / 2))
if n_fft % 2:
    n_points += 1
freq = fs/2 * np.linspace(-1, 1, n_points)

# X_w_norm = np.abs(fftshift(X_w))
X_w_norm = 20 * np.log10(np.abs(fftshift(X_w / abs(X_w).max())))

a = np.diff(np.sign(np.diff(X_w_norm))).nonzero()[0] + 1               # local min & max
b = (np.diff(np.sign(np.diff(X_w_norm))) > 0).nonzero()[0] + 1         # local min
c = (np.diff(np.sign(np.diff(X_w_norm))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number

plt.plot(freq, X_w_norm, color="grey")
plt.plot(freq, [-10 for _ in X_w_norm], color="orange")
plt.plot(freq[b], X_w_norm[b], "o", label="min", color='r')
plt.plot(freq[c], X_w_norm[c], "o", label="max", color='b')
plt.title("Frequency response first activity")
plt.ylabel("Normalized magnitude [dB]")
plt.xlabel("F [Hz]")
plt.xlim([0, 5])
print(act_tp_df.loc[0, "label"])
plt.show()


Let's build a function that pulls out the max of the peaks that cross the threshold (i.e., just gets the 0.02, 0.95, and 2.73)

In [ ]:
def local_fmax_above_thresh(freq, x_w, threshold):
    local_max_ix = (np.diff(np.sign(np.diff(x_w))) < 0).nonzero()[0] + 1
    x_w_max = x_w[local_max_ix]
    freq_max = freq[local_max_ix]

    return freq_max[np.where((x_w_max>threshold) & (freq_max>0))]

In [ ]:
local_fmax_above_thresh(freq, X_w_norm, -10)

Now, we want to cycle through all of the activities, and extract the peak frequencies above a particular threshold. What I want to find is an "ideal" threshold such that I'm only picking out 2 peak frequencies (3 including 0 Hz) for the majority of activities. This will be the threshold I work with for the rest of the project to extract features.

In [ ]:
def calculate_normed_spectrum(df, fs=50):
    n_fft = df.shape[0]
    window = signal.hann(n_fft)
    X_w = fft(window * df.accel_x.values)
    X_w_norm = 20 * np.log10(np.abs(fftshift(X_w / abs(X_w).max())))

    n_points = 2 * int(np.floor(n_fft / 2))
    if n_fft % 2:
        n_points += 1
    freq = fs/2 * np.linspace(-1, 1, n_points)
    return X_w_norm, freq

In [ ]:
thresholds = [-5, -10, -15, -20]
all_pks_df = pd.DataFrame(columns=["threshold", "activity", "peak_fs"])
for thresh in thresholds:
    for r_ix in range(activity_starts_df.shape[0]):
        act_tp_df = pd.concat([activity_starts_df.loc[[r_ix], :], activity_ends_df.loc[[r_ix], :]], ignore_index=True)
        df_snip = df_dropna[(df_dropna.time >= act_tp_df.loc[0, "time"]) & (df_dropna.time <= act_tp_df.loc[1, "time"])]
        X_w_norm, freq = calculate_normed_spectrum(df_snip)

        local_fmax = local_fmax_above_thresh(freq, X_w_norm, thresh)
        data = {
            "threshold": [thresh for _ in local_fmax],
            "activity": [act_tp_df.loc[0, "label"] for _ in local_fmax],
            "peak_fs": local_fmax
        }
        pks_df = pd.DataFrame(data=data)
        all_pks_df = pd.concat([all_pks_df, pks_df], ignore_index=True)

In [ ]:
all_pks_df["q_rounded_fs"] = all_pks_df["peak_fs"].apply(lambda x: np.round(x * 4) / 4)

In [ ]:
all_pks_df.q_rounded_fs.unique()

In [ ]:
all_pks_df["h_rounded_fs"] = all_pks_df["peak_fs"].apply(lambda x: np.round(x * 2) / 2)
all_pks_df.h_rounded_fs.unique()

Seems like simply going to full-rounded frequencies may be the way to go to limit the total number of features I'll need to generate across activities. I'll test it across datasets to verify.

In [ ]:
all_pks_df["rounded_fs"] = all_pks_df["peak_fs"].apply(lambda x: np.round(x))
np.sort(all_pks_df.rounded_fs.unique())

But first, I should separate out a test dataset from the set of raw parquet files. There are 126 files, and we want an 80/20 split, so `126 * 0.2 ~= 25` files should be designated for the test set. There are also 94 subjects, and some users should only be included in the test set (to better ensure generalizability of the models), so `94 * 0.2 ~= 19` subjects should be included in the test set. In order to achieve this split, I will find 13 subjects with 1 exercise run, and 6 subjects with 2 exercise runs. This will give me `13 * (1) + 6 * (2) = 13 + 12 = 25` total files in the test set.

In [ ]:
parquet_dir = Path("../../data/interim/raw/")
all_files = list(x for x in parquet_dir.iterdir() if x.is_file())

In [ ]:
def _make_files_dict(all_files):
    files_dict = defaultdict(list)
    for file in all_files:
        _, subj, _ = file.parts[-1].split("_")
        files_dict[subj].append(str(file))
    
    return files_dict

def _get_first_subjs_match_crit(files_dict, n_sing_file, n_double_file):
    test_subj_ids = []
    ones_left = n_sing_file
    twos_left = n_double_file
    for key, val in files_dict.items():
        if not (ones_left or twos_left):
            break

        if (len(val) == 1) & (ones_left > 0):
            test_subj_ids.append(key)
            ones_left -= 1
        elif (len(val) == 2) & (twos_left > 0):
            test_subj_ids.append(key)
            twos_left -= 1

    return test_subj_ids

def _make_train_test_dict(all_files, test_subj_ids):
    train_test_files = defaultdict(list)
    for file in all_files:
        _, subj, _ = file.parts[-1].split("_")
        if any([subj==test_subj for test_subj in test_subj_ids]):
            train_test_files["test"].append(str(file))
        else:
            train_test_files["train_val"].append(str(file))
    
    return train_test_files

def make_train_test_split_json(interim_path):
    all_files = list(x for x in interim_path.iterdir() if x.is_file())
    files_dict = _make_files_dict(all_files)
    
    test_subj_ids = _get_first_subjs_match_crit(files_dict, 13, 6)

    train_test_files = _make_train_test_dict(all_files, test_subj_ids)
    
    with open("../../src/data/train-val_test.json", "w", encoding="utf-8") as outfile:
        json.dump(train_test_files, outfile)
    
    return files_dict, test_subj_ids

In [ ]:
subj_files, test_subj_ids = make_train_test_split_json(parquet_dir)

Assertions check out, so we have our list of test subjects, for which all fileIDs will be included in the test dataset.

In [ ]:
n_files = sum([len(subj_files[key]) for key in test_subj_ids])
assert len(test_subj_ids) == 19
assert n_files == 25

Now, let's use this same methodology to split the training/validation files into strict training vs. validation datasets. We'll use the same 80/20 split. Given that there are 101 files, we want `101 * 0.2 ~= 20` files saved for the validation set. Additionally, since there are 75 subjects in the train/val set, we want `75 * 0.2 = 15` subjects in the validation set. I will attempt to proportionally split out the number of subjects with large (i.e., more than 1) data sets by hand based on this split, and then fill in the rest.

In [ ]:
def make_train_val_split_json(train_val_test_path, n_val_subjs, n_val_files, n_files_tol=1, **kwargs):
    default_ns = {
        "n_5_files": 0,
        "n_4_files": 1,
        "n_3_files": 1,
        "n_2_files": 1 
    }
    val_ns_files = {
        key: val if key not in kwargs.keys() else kwargs[key]
        for key, val in default_ns.items()
    }
    val_ns_files.update(
        {key: val for key, val in kwargs.items() if key not in val_ns_files.keys()}
    )

    with open(train_val_test_path, "r", encoding="utf-8") as infile:
        train_test_files = json.load(infile)

    train_val_files = [Path(file) for file in train_test_files["train_val"]]
    
    files_dict = _make_files_dict(train_val_files)


    ns_dict = {
        key: len(val) for key, val in files_dict.items()
    }
    sort_subjs_ns = sorted(ns_dict.items(), key=lambda x: x[1], reverse=True)
    sort_subjs, sorted_ns = (
        [s[0] for s in sort_subjs_ns],
        [s[1] for s in sort_subjs_ns],
    )
    val_subjs, val_ns = [], []
    for key, desired_count in val_ns_files.items():
        desired_n = int(key.split("_")[1])
        for n in range(desired_count):
            try:
                n_ix = sorted_ns.index(desired_n)
            except ValueError:
                raise ValueError((
                    f"Couldn't find subject {n} with desired count {desired_count} in "
                    "the files left over for validation split. Please reduce the requested"
                    " number of subjects for this desired count and re-run."
                ))

            val_subjs.append(sort_subjs.pop(n_ix))
            val_ns.append(sorted_ns.pop(n_ix)) 
            n_val_files -= val_ns[-1]

    while n_val_files>0:
        val_subjs.append(sort_subjs.pop())
        n_val_files -= sorted_ns.pop()

    if len(val_subjs)>n_val_subjs:
        print("Too many subjects selected based in initial passthrough. Attempting to fix...")
        while n_files_tol>0:
            # popping from end, which removes subjects with 1 data file first
            sort_subjs.append(val_subjs.pop())
            sorted_ns.append(val_ns.pop())
            n_files_tol -= sorted_ns[-1]

        if len(val_subjs)>n_val_subjs:
            raise ValueError((
                f"Cannot reconcile requirements for validation subject counts having "
                f"particular numbers of data files with the constraints on number of "
                f"validation subjects {n_val_subjs}, number of total validation files "
                f"{n_val_files} and validation file count tolerance {n_files_tol}. Please"
                f" either adjust subject counts by number of data files, the total number"
                f" of desired validation files, or increase file count tolerance."))
        else:
            print("Fixed total-validation subject constraint given file-count tolerance...")
    elif len(val_subjs)<n_val_subjs:
        print("Not enough subjects selected based in initial passthrough. Attempting to fix...")

        while n_files_tol>0:
            # popping from end, which removes subjects with 1 data file first
            val_subjs.append(sort_subjs.pop())
            val_ns.append(sorted_ns.pop()) 
            n_files_tol -= val_ns[-1]

        if len(val_subjs)>n_val_subjs:
            raise ValueError((
                f"Cannot reconcile requirements for validation subject counts having "
                f"particular numbers of data files with the constraints on number of "
                f"validation subjects {n_val_subjs}, number of total validation files "
                f"{n_val_files} and validation file count tolerance {n_files_tol}. Please"
                f" either adjust subject counts by number of data files, the total number"
                f" of desired validation files, or increase file count tolerance."))
        else:
            print("Fixed total-validation subject constraint given file-count tolerance...")

    # now, find the file names for each subject in val_subjs and sort_subjs (which is now
    # just training subjects) and store as dict
    train_val_dict= {
        "validation": [file for subj in val_subjs for file in files_dict[subj]],
        "train": [file for subj in sort_subjs for file in files_dict[subj] ],
    }


    # now, write dict to json
    with open("../../src/data/train_val.json", "w", encoding="utf-8") as outfile:
        json.dump(train_val_dict, outfile)

    return train_val_dict
    

In [ ]:
n_files_tol=1
desired_val_subjs = 15
desired_val_files = 20
train_val_dict = make_train_val_split_json("../../src/data/train-val_test.json", desired_val_subjs, desired_val_files, n_files_tol=n_files_tol)

Let's check that the data is split correctly. 

(1) If we want 1 subject each with 4, 3, and 2 data files, respectively, then there should be:
- 1 data file with "dataID3" in the file name,
- two data files with "dataID2" in the file name, and
- three data files with "dataID1" in the file name

(2) there should also be data from 15 subjects in the validation set.

(3) there should be 20 (+/- 1) total validation files (3).

(4) there should be no overlap between files in validation and training

(5) there should be no duplicate files in training

In [ ]:
# 1
nd3 = sum("dataID3" in file for file in train_val_dict["validation"])
nd2 = sum("dataID2" in file for file in train_val_dict["validation"])
nd1 = sum("dataID1" in file for file in train_val_dict["validation"])

# these checks will need to be manually changed if we change the default values for 
# kwargs in make_train_val_split_json()
assert nd3==1, "Too many subjects with 4 data files"
assert nd2==2, "Too many subjects with 3 data files"
assert nd1==3, "Too many subjects with 2 data files"

# 2
patt = re.compile("(subjID\d+_)")
n_subjs = len(set([patt.findall(file)[0] for file in train_val_dict["validation"]]))

assert n_subjs==desired_val_subjs, "Too many subjects in validation set"

# 3
n_files = len(train_val_dict["validation"])
assert abs(n_files - desired_val_files) <= n_files_tol, "Total validation file count not within tolerance of desired number"

# 4
n_overlap = sum(
    [val_file == train_file for val_file in train_val_dict["validation"] for train_file in train_val_dict["train"]]
)
assert n_overlap==0, "Overlapping files between train and validations sets"

# 5
n_dupl = sum(
    [
        train_val_dict["train"][ix1] == train_val_dict["train"][ix2] 
        for ix1 in range(len(train_val_dict["train"])) 
        for ix2 in range(ix1+1, len(train_val_dict["train"]))
    ]
)
assert n_dupl==0, "Duplicated file names in training set"

Assertions check out, so we are good to move forward...